In [ ]:
import os
import cv2
import mediapipe as mp
import winsound 

# Webcam setup
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Background image
imgBackground = cv2.imread(r"C:\Users\ELLURI BHARGAVI\Downloads\Resources\Resources\Background.png")

# Importing mode images
folderPathModes = r"C:\Users\ELLURI BHARGAVI\Downloads\Resources\Resources\Modes"
listImgModesPath = os.listdir(folderPathModes)
listImgModes = [cv2.imread(os.path.join(folderPathModes, imgModePath)) for imgModePath in listImgModesPath]

# Importing icons
folderPathIcons = r"C:\Users\ELLURI BHARGAVI\Downloads\Resources\Resources\Icons"
listImgIconsPath = os.listdir(folderPathIcons)
listImgIcons = [cv2.imread(os.path.join(folderPathIcons, imgIconsPath)) for imgIconsPath in listImgIconsPath]

# Selection variables
modeType = 0
selection = -1
counter = 0
selectionSpeed = 7
modePositions = [(1136, 196), (1000, 384), (1136, 581)]
counterPause = 0
selectionList = [-1, -1, -1]

# MediaPipe Hands setup
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.8)
mpDraw = mp.solutions.drawing_utils

def fingersUp(hand_landmarks):
    """Return list of finger states [Thumb, Index, Middle, Ring, Pinky]"""
    tipIds = [4, 8, 12, 16, 20]
    fingers = []

    # Thumb: compare x coordinates (for right hand)
    if hand_landmarks.landmark[tipIds[0]].x < hand_landmarks.landmark[tipIds[0] - 1].x:
        fingers.append(1)
    else:
        fingers.append(0)

    # Other fingers: compare y coordinates
    for id in range(1, 5):
        if hand_landmarks.landmark[tipIds[id]].y < hand_landmarks.landmark[tipIds[id] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    # Overlay webcam feed
    imgBackground[139:139 + 480, 50:50 + 640] = img
    imgBackground[0:720, 847:1280] = listImgModes[modeType]

    if results.multi_hand_landmarks and counterPause == 0 and modeType < 3:
        handLms = results.multi_hand_landmarks[0]
        fingers1 = fingersUp(handLms)
        print(fingers1)

        if fingers1 == [0, 1, 0, 0, 0]:
            if selection != 1: counter = 1
            selection = 1
        elif fingers1 == [0, 1, 1, 0, 0]:
            if selection != 2: counter = 1
            selection = 2
        elif fingers1 == [0, 1, 1, 1, 0]:
            if selection != 3: counter = 1
            selection = 3
        else:
            selection = -1
            counter = 0

        if counter > 0:
            counter += 1
            cv2.ellipse(imgBackground, modePositions[selection - 1], (103, 103), 0, 0,
                        counter * selectionSpeed, (0, 255, 0), 20)
            if counter * selectionSpeed > 360:
                selectionList[modeType] = selection
                modeType += 1
                counter = 0
                selection = -1
                counterPause = 1
                winsound.Beep(2000,500)

    # Pause logic
    if counterPause > 0:
        counterPause += 1
        if counterPause > 60:
            counterPause = 0

    # Add icons
    if selectionList[0] != -1:
        imgBackground[636:701, 133:198] = listImgIcons[selectionList[0] - 1]
    if selectionList[1] != -1:
        imgBackground[636:701, 340:405] = listImgIcons[2 + selectionList[1]]
    if selectionList[2] != -1:
        imgBackground[636:701, 542:607] = listImgIcons[5 + selectionList[2]]

    cv2.imshow("Background", imgBackground)
    if cv2.waitKey(1) & 255 == ord('q') :
        break 

cap.release() 
cv2.destroyAllWindows() 